In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ChromeOptions
import time
import pickle
import re
from unidecode import unidecode

In [ ]:
chrome_options = ChromeOptions()
chrome_options.binary_location = '''C:/Program Files/Chromium/chrome.exe'''
chrome_options.add_argument('headless')

In [ ]:
def rankScrape(n):
    cont = []
    for i in range(n):
        pn = str(i+1)
        url = f"https://boardgamegeek.com/browse/boardgame/page/{pn}"
        driver = webdriver.Chrome(service=Service('chromedriver.exe'), options=chrome_options)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,"html5lib")
        stopper = False
        row = 0
        for row in soup.find_all('tr', id = 'row_'):
            row += 1
            rank = row.find('td', class_='collection_rank').text.strip()
            if rank == "N/A": #oops
                stopper = True
                break
            title = row.find('div', id = f'results_objectname{row}').find('a').text.strip() 
            link = row.find('div', id = f'results_objectname{row}').find('a').get('href').strip()
            fulllink = f"https://boardgamegeek.com{link}"          
            
            data = {'rank':rank,'title':title,'gamepage':fulllink}
            cont.append(data)           
        driver.quit()
        if stopper:
            break
        # time.sleep(1)
    return pd.DataFrame(cont)

In [300]:
# df_list = rankScrape(10)
# pickle.dump(df_list, open('dflist.p', 'wb'))
df_list = pickle.load(open('./dflist.p', 'rb'))

In [301]:
df_list

,rank,title,gamepage
0,1,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...
1,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...
2,3,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...
3,4,Gloomhaven: Jaws of the Lion,https://boardgamegeek.com/boardgame/291457/glo...
4,5,Terraforming Mars,https://boardgamegeek.com/boardgame/167791/ter...
...,...,...,...
995,996,Little Town,https://boardgamegeek.com/boardgame/241266/lit...
996,997,Mr. Jack in New York,https://boardgamegeek.com/boardgame/55427/mr-j...
997,998,Linko!,https://boardgamegeek.com/boardgame/153065/linko
998,999,Smash Up,https://boardgamegeek.com/boardgame/122522/smash


In [307]:
# (colname, regexstring)
stuff = [("title", '''"name":"(['\$\#/a-zA-Z\s0-9\-:!\&\,\.(?\\\\u)`\(\)\\\\]+)","alternatename'''),
("year", '''"yearpublished":"(\-?\d{1,4})'''),
("minP_r", '''minplayers":"(\d+)"'''),
("maxP_r", '''maxplayers":"(\d+)"'''),
("minP_u", '''"recommended":\[\{"min":(\d+)'''),
("maxP_u", '''"recommended":\[\{"min":\d+,"max":(\d+)\}\]'''),
("minT_r", '''"minplaytime":"(\d+)"'''),
("maxT_r", '''"maxplaytime":"(\d+)"'''),
("age_r", '''"minage":"(\d+)"'''),
("desc", '''"short\_description":"([a-zA-Z0-9\s\-:;!\&,.'\?(?\\\\u)]+)"'''),
("primDesign", '''\{"boardgamedesigner":\[\{"name":"([a-zA-Z!\s\-(?\\\\u)0-9\.'\,]+)"'''),
("weight", '''"averageweight":(\d\.?\d{0,2})'''),
("numRatings", '''"stats":\{"usersrated":"(\d+)"'''),
("rating", '''"stats":\{"usersrated":"\d+","average":"(\d\.\d{1,3})'''),
("geekrating", '''"stats":\{"usersrated":"\d+","average":"\d\.\d+","baverage":"(\d\.\d{3})'''),
("owned", '''"numowned":"(\d+)"'''),
("wishlist", '''"numwish":"(\d+)"'''),
("prev_owned", '''numprevowned":"(\d+)"'''),
("comments", '''"numcomments":"(\d+)"'''),
("plays_total", '''"numplays":"(\d+)"'''),
("designers", '''"name":"([a-zA-Z!\s\-(?\\\\u)0-9\.'\,]+)","objecttype":"person",[\d\w:,\\\\/"]*boardgamedesigner'''),
("categories", '''"name":"([a-zA-Z\s\-]+)","objecttype":"property",[\d\w:,\\\\/"]*boardgamecategory'''),
("mechanics", '''"name":"([a-zA-Z\s\-]+)","objecttype":"property",[\d\w:,\\\\/"]*boardgamemechanic''')]

In [310]:
_cont = []
for index, row in df_list.iterrows():
    x = requests.get(row['gamepage'])
    y = BeautifulSoup(x.text, 'html5')
    z = y.find('script', text=re.compile(r'(GEEK\.geekitemPreload.*)'))
    newdict={}
    for i in stuff:
        try:
            if i[0] in ['designers', 'categories', 'mechanics']:
                newdict[i[0]] = re.findall(re.compile(i[1]), z.contents[0]) # multiples
            elif i[0] == 'maxP_u':
                try:
                    newdict[i[0]] = re.findall(re.compile(i[1]), z.contents[0])[0]
                except:
                    continue
            else: 
                newdict[i[0]] = re.findall(re.compile(i[1]), z.contents[0])[0] # single value
        except:
            print(row['title'], i[0], re.findall(re.compile(i[1]), z.contents[0]), "\n")
    _cont.append(newdict)
    if index % 100 == 0:
        time.sleep(1)
    # if index==50:
        # break
df = pd.DataFrame(_cont)
# df.info()

Power Grid Deluxe: Europe/North America desc [] 

Blood Bowl: Living Rulebook desc [] 

Blood Bowl (Third Edition) desc [] 

Raptor desc [] 

Madeira desc [] 

No Thanks! desc [] 

Star Wars: X-Wing Miniatures Game – The Force Awakens Core Set desc [] 

Love Letter: Batman desc [] 

The Networks desc [] 

Fugitive desc [] 

Union Pacific desc [] 

Age of Industry desc [] 

Space Hulk (Fourth Edition) primDesign [] 

Zombicide Season 3: Rue Morgue desc [] 

Unlock!: Epic Adventures desc [] 

Little Town desc [] 



In [311]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1000 non-null   object
 1   year         1000 non-null   object
 2   minP_r       1000 non-null   object
 3   maxP_r       1000 non-null   object
 4   minP_u       1000 non-null   object
 5   maxP_u       935 non-null    object
 6   minT_r       1000 non-null   object
 7   maxT_r       1000 non-null   object
 8   age_r        1000 non-null   object
 9   desc         985 non-null    object
 10  primDesign   999 non-null    object
 11  weight       1000 non-null   object
 12  numRatings   1000 non-null   object
 13  rating       1000 non-null   object
 14  geekrating   1000 non-null   object
 15  owned        1000 non-null   object
 16  wishlist     1000 non-null   object
 17  prev_owned   1000 non-null   object
 18  comments     1000 non-null   object
 19  plays_total  1000 non-null  

In [ ]:
pickle.dump(df, open('df.p', 'wb'))